Todo:
- spin down handel resource does not exist error
- error - cannot delete policy / most delete policy to delete role

In [15]:
import boto3
import os
import zipfile
import json
from pprint import pprint
from botocore.exceptions import ClientError
import io
import time

In [16]:
def lambda_exists(function_name):
    try:
        function = lambda_client.get_function(FunctionName=function_name)
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            return False
        raise Exception(e)


def list_functions():
    for f in lambda_client.list_functions()['Functions']:
        print(f['FunctionName'])


def delete_role(role_name, delete_policies: bool = True):
    policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
    for policy in policies:
        iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
        if delete_policies:
            try:
                iam_client.delete_policy(PolicyArn=policy['PolicyArn'])
            except ClientError as e:
                print(f'Cannot delete: {policy["PolicyName"]}')
    iam_client.delete_role(RoleName=role_name)


def get_policy(policy_name):
    result = [i for i in iam_client.list_policies()['Policies'] if i['PolicyName'] == policy_name][0]
    return iam_client.get_policy(PolicyArn=result['Arn'])
    

def delete_policy(policy_name):
    for policy in iam_client.list_policies(Scope='Local')['Policies']:
        if policy['PolicyName'] == policy_name:
            iam_client.delete_policy(PolicyArn=policy['Arn'])
            break


def spin_down_resources() -> None:
    scheduler_client.delete_schedule(Name=scheduler_name)
    delete_role(role_name=invoke_role_name)
    delete_role(role_name=role_name)
    lambda_client.delete_function(FunctionName=function_name)
    delete_policy(policy_name=invoke_policy_name)
    delete_policy(policy_name=s3_put_policy)
    


def delete_bucket(bucket_name):
    objects = s3_client.list_objects(Bucket=bucket_name).get('Contents', [])
    for obj in objects:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        print(f'Deleted object: {obj["Key"]}')
    s3_client.delete_bucket(Bucket=bucket_name)


def bucket_exists(bucket_name):
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        return True
    except ClientError as e:
        if e.response['Error']['Message'] == 'Not Found':
            return False
        else:
            raise e

            
def delete_objects(bucket_name):
    for o in s3_client.list_objects(Bucket=bucket_name)['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=o['Key'])
        print(f'Delete object: {key}')

Defining the clients and names

In [17]:
aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID']
aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY']
aws_region = os.environ['AWS_REGION']
credentials = {'region_name':aws_region, 'aws_access_key_id':aws_access_key_id, 'aws_secret_access_key':aws_secret_access_key}

lambda_client = boto3.client('lambda', **credentials)
events_client = boto3.client('events', **credentials)
iam_client = boto3.client('iam', **credentials)
scheduler_client = boto3.client('scheduler', **credentials)
s3_client = boto3.client('s3', **credentials)

# Replace these values with your own
function_name = 'HelloWorldLambda'
role_name = 'HelloWorldLambdaRole'
invoke_role_name = 'InvokeLambdaRole'
invoke_policy_name = 'InvokeLambdaPolicy'
scheduler_name = 'LambdaEveryMinuteSchedule'
bucket_name = 'hello-world-bucket-lambda'
s3_put_policy = 'S3PutObjectPolicy'

### Create bucket

In [18]:
if not bucket_exists(bucket_name):
    bucket = s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': aws_region})
else:
    print(f'Bucket already exists: {bucket_name}')

Bucket already exists: hello-world-bucket-lambda


### Create lambda

Create lambda role and policies

In [19]:
assume_role_policy_document = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {
                'Service': 'lambda.amazonaws.com'
            },
            'Action': 'sts:AssumeRole'
        }
    ]
}


try:
    role = iam_client.get_role(RoleName=role_name)['Role']
except Exception:
    role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
    )['Role']
    time.sleep(10)

res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)


In [20]:
s3_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3:PutObject",
            "Resource": f"arn:aws:s3:::{bucket_name}/*"
        }
    ]
}

res = iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName=s3_put_policy,
    PolicyDocument=json.dumps(s3_policy_document)
)

Create lambda

In [21]:
with open('lambda_function.py', 'r') as f:
    lambda_py = f.read().format(bucket_name=bucket_name)
in_memory_zip = io.BytesIO()
with zipfile.ZipFile(in_memory_zip, 'w', zipfile.ZIP_DEFLATED) as f:
 f.writestr('lambda_function.py', lambda_py)
in_memory_zip.seek(0)

if lambda_exists(function_name):
    lambda_client.delete_function(FunctionName=function_name)
    print(f'deleted existing lambda: {function_name}')


lambda_response = lambda_client.create_function(
    FunctionName=function_name,
    Runtime='python3.8',
    Role=role['Arn'],
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': in_memory_zip.read()},
    Timeout=10,
    MemorySize=128
)

deleted existing lambda: HelloWorldLambda


### Create schedule

In [22]:
policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": ["lambda:InvokeFunction"],
            "Resource": [
                lambda_response['FunctionArn'],
                lambda_response['FunctionArn'] + ':*'
            ]
        }
    ]
}

try:
    policy = get_policy(invoke_policy_name)
except (ClientError, IndexError) as e:
    policy = iam_client.create_policy(
        PolicyName=invoke_policy_name,
        PolicyDocument=json.dumps(policy_document)
    )

In [23]:
try:
    invoke_role = iam_client.get_role(RoleName=invoke_role_name)
except ClientError as e:
    assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "scheduler.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
    }
    invoke_role = iam_client.create_role(RoleName = invoke_role_name, AssumeRolePolicyDocument=json.dumps(assume_role_policy))
    time.sleep(10)

res = iam_client.attach_role_policy(RoleName=invoke_role_name, PolicyArn=policy['Policy']['Arn'])


In [24]:
try:
    scheduler = scheduler_client.get_schedule(Name=scheduler_name)
except ClientError as e:
    scheduler = scheduler_client.create_schedule(
        Name=scheduler_name,
        ScheduleExpression='rate(1 minute)',
        State='ENABLED',
        Target={
            'Arn': lambda_response['FunctionArn'],
            'RoleArn': invoke_role['Role']['Arn']
        },
        FlexibleTimeWindow={
            'Mode': 'OFF'
        }
    )

In [25]:
spin_down_resources()

Cannot delete: AWSLambdaBasicExecutionRole


DeleteConflictException: An error occurred (DeleteConflict) when calling the DeleteRole operation: Cannot delete entity, must delete policies first.